In [ ]:
# import requests



# def download_file(url, save_path):
#     response = requests.get(url)
#     with open(save_path, 'wb') as file:
#         file.write(response.content)
#     print("Downloaded successfully!")


# from bs4 import BeautifulSoup
# import urllib.request

# parser = 'html.parser'

# url = "https://bwsyncandshare.kit.edu/s/ooNSgk2aN9opB5S"


# resp = urllib.request.urlopen(url)
# soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

# all_links = list()

# for link in soup.find_all('a', href=True):
#     print(link['href'])

#     valid_link_name = (".tar" in link["href"]) or ("/download" in link["href"])

#     if valid_link_name and link["href"] not in all_links:
#         print("YEA")
#         format = link["href"].split(".")[-1]
#         name = link["href"].split(".")[-2].split("/")[-1]

#         if format not in ["zip", "tar", "csv", "parquet", "avro", "json"]:
#             format = "tar"
#         download_file(link["href"], f"D:/{name}.{format}")

#     all_links.append(link["href"])


# import tarfile

# if format == "tar":

#     with tarfile.open(f"D:/{name}.{format}", "r") as tar:
#                         df_list = []
#                         for file in tar.getmembers():
#                             # extract the DAT file
#                             dat_data = tar.extractfile(file)
                

In [ ]:
from Preprocessing_M import Data_Extractor, Data_Preprocessing

de = Data_Extractor("C:/Users/danie/hka-aqm-data", "C:/Users/danie/hka-aqm-data/hka-aqm-data")
directory = "C:/Users/danie/hka-aqm-data/hka-aqm-data"

de.extract_zip_files(directory)
de.delete_zip_files(directory)

df = de.get_data(directory)

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
df.columns

In [ ]:
df.loc[:, "room_number"] = df["device_id"].str.split("-").str[-1]
df.loc[:, "building_name"] = df["room_number"].str.replace('\d+', '', regex = True)
df_m = df[df.building_name == "m"]

In [ ]:
df_m.dtypes

In [ ]:
df_m["snr"] = df_m["snr"].astype(float)
df_m["date_time"] = pd.to_datetime(df_m["date_time"])

Remove columns with only one unique value

In [ ]:
for col in df_m.columns:
    if df_m[col].nunique() <= 1:
        df_m.drop(columns = [col], axis = 1, inplace = True)
        print(f"Removed column: {col}")

In [ ]:
df_m.head()

## Temperature (tmp) ## 

In [ ]:
px.box(df_m.tmp, title = "Boxplot of measured temperatures (°C) in building M")

In [ ]:
df_m[df_m.tmp > 50]

In [ ]:
df_m = df_m[df_m.tmp <= 50]

In [ ]:
px.box(df_m.tmp, title = "Boxplot of measured temperatures (°C) in building M (after cleaning)")

## Relative air humidity ##

In [ ]:
px.box(df_m.hum, title = "Boxplot of measured relative air humidity (in %) in building M")

In [ ]:
df_m[df_m.hum > 100][["date_time", "room_number", "CO2", "VOC", "tmp", "hum"]]

In [ ]:
df_m = df_m.loc[df_m.hum <= 100, :]
px.box(df_m.hum, title = "Boxplot of measured relative air humidity (in %) in building M (after cleaning invalid values)")

Remove data points with zeroes as values.

In [ ]:
values_zero = (df_m.tmp == 0) | (df_m.hum == 0) | (df.CO2 == 0) | (df.VOC == 0)

df_m = df_m[values_zero == False]

## CO2 and VOC ##

In [ ]:
import seaborn as sns
sns.heatmap(df_m[["CO2", "VOC", "tmp", "hum", "IR", "vis"]].corr())

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot(df_m.CO2)

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot(df_m.VOC)

In [ ]:
df_m[df_m.CO2 > 3000].room_number.unique()

In [ ]:
px.line(df_m[df_m.room_number == "m001"][["date_time", "CO2"]], x = "date_time", y = "CO2")

In [ ]:
df_m = df_m.sort_values(by = "date_time")

df_m["CO2_diff"] = df_m.groupby("room_number").CO2.diff()
df_m["VOC_diff"] = df_m.groupby("room_number").VOC.diff()
df_m["time_diff"] = df_m.groupby("room_number").date_time.diff().dt.seconds
df_m.fillna(0, inplace = True)

## Room number ##

In [ ]:
df_m.loc[:, "date_time"] = pd.to_datetime(df_m["date_time"])
df_m.loc[:, "year"] = df_m.date_time.dt.year.astype(str)
px.bar(df_m.groupby(["year", "room_number"]).agg(data_points = ("CO2", "count")).reset_index(), x = "room_number", y = "data_points", color = "year", barmode = "group", title = "Data points per room in building M")

In [ ]:
df_m[df_m.room_number == "m700"]

In [ ]:
df_m.to_parquet("C:/Users/danie/hka-aqm-data-m.parquet")

In [ ]:
# from Preprocessing_M import Data_Preprocessing

# dp = Data_Preprocessing(get_outliers_out = True)

# df = dp.preprocess_df(df)